**Exemplo simples de leitura por meio de socket**

In [1]:
#cria a seção a ser utiliza para estabelecer a conexão 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window
from pyspark.sql.functions import col


spark = SparkSession \
    .builder \
    .appName("StructuredWorkWordCount") \
    .getOrCreate()

In [2]:
#cria o dataframe que será responsável por ler cada uma das linhas recebidas através do localhost e porta 9999
# define a fonte (source) de dados
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Divide as linhas recebidas em cada palavra
words = lines.select(
        explode(
            split(lines.value, ' ')
        ).alias('word')
    )

# cria o novo dataframe a ser responsável por contar a quantidade de palavras digitadas
wordCounts = words.groupBy("word").count()

In [3]:
#confere o tipo de dataframe
wordCounts.isStreaming

True

In [ ]:
# Define a consulta (query) e como deve ser realizada a saída (sink) para o stream criado
query = wordCounts\
    .writeStream\
    .outputMode("update")\
    .format("console")\
    .start()
#opções válidas para outputMode: append, complete, update

query.awaitTermination() #aguarda até que a streaming query termine

Para rodar é necessário ter o "nc" instalado e utiliar o comando antes de iniciar a célula anterior

nc -lk 9999